#CS 5350 - Homework 1

##Task and Data
Used the CSV training data to construct a tree

Struggled to understand how to actually use my tree for classification and validation at an algorithmic level.

In [74]:
#Unzip, extract
import numpy as np
import pandas as pd
import csv
eps = np.finfo(float).eps
from numpy import log2 as log
import pprint
#other dependecies

#Full Tree Implementation

In [76]:
# finds the best
def findMaxIG(dict):
  return max(dict.values()), max(dict, key=dict.get)

In [77]:
#entropy
def entropy(df):
  h = 0
  label_vals=df[0].unique()

  for value in label_vals:
    ratio = df[0].value_counts()[value]/len(df[0])
    h += -ratio*log(ratio)
    
  return h

In [78]:
def feature_entropy(df,feature):
  target_variables = df[0].unique()
  variables = df[feature].unique() #gives 0, 1

  sum_h = 0

  for variable in variables:
    h = 0

    for target_var in target_variables:
      num_target = len(df[feature][df[feature]==variable][df[0] == target_var])
      num_total = len(df[feature][df[feature]==variable])
      ratio = num_target/(num_total+eps)
      h += -ratio*log(ratio+eps)

    ratio2 = num_total / len(df)
    sum_h += -ratio2*h

  return abs(sum_h)

In [79]:
def ig_max(df):
  h_feat = []
  ig_arr = []
  h=entropy(df)
  
  for key in df.keys()[1:]:
    ig = h-feature_entropy(df,key)
    ig_arr.append(ig)

  max_index = np.argmax(ig_arr)+1
  max_val = ig_arr[max_index-1]
  return max_index, max_val

In [80]:
def get_examples(df, feature, value):
  return df[df[feature] == value].reset_index(drop=True)

In [144]:
def make_tree(df, tree=None):
  feature, ig_val = ig_max(df)

  feat_value = np.unique(df[feature])

  if tree is None:
    tree={}
    tree[feature]={}

  for value in feat_value:
    examples = get_examples(df, feature, value)
    clValue,counts = np.unique(examples[value],return_counts=True)

    if len(counts) ==1:
      tree[feature][value] = clValue[0]
    else:
      tree[feature][value] = make_tree(examples)
    

  return tree



In [243]:
#df playground
df = pd.read_csv("a1a_train.csv", header=None)

print(f"Entropy of Dataset: {entropy(df)}")

max_index, max_ig = ig_max(df)
print(f"IG of Root node: {max_ig}, Feature: {max_index}")

t = make_tree(df)

Entropy of Dataset: 0.8025159515884955
IG of Root node: 0.1539453347407419, Feature: 40


##Tree Visualization

In [244]:
pprint.pprint(t)

{40: {0: {39: {0: {74: {0: {6: {0: {49: {0: 1, 1: 0}}, 1: {81: {0: -1, 1: 0}}}},
                        1: {51: {0: {27: {0: {42: {0: {11: {0: {82: {0: {52: {0: -1,
                                                                              1: {3: {0: -1,
                                                                                      1: 0}}}},
                                                                     1: {25: {0: {43: {0: -1,
                                                                                       1: {4: {0: -1,
                                                                                               1: 0}}}},
                                                                              1: 0}}}},
                                                            1: 0}},
                                                   1: {13: {0: -1, 1: 1}}}},
                                          1: {7: {0: {82: {0: -1,
                                             

#Full Tree Implementation Evaluation

In [245]:
#evaluation
def evaluate(tree, example, i):

  #base case are we at the end?
  if isinstance(tree, np.int64):  
    return tree == example[i][0]

  #assuming we hit another dictionary
  for k,v in tree.items():
    if isinstance(v, dict):      
      #check key against example
      #to find what path to take
      path = example[i][k]
      
      return evaluate(v[path], example, i)

In [246]:
#Testing the Full Tree Implementation
nparr = df.to_numpy()

correct = 0

for i in range(nparr.shape[0]):
  testVal = evaluate(t, nparr, i)
  #print(f"evaluated: {testVal}")
  if testVal == True:
    correct += 1

print("Evaluate Training Data")
print(f"total correct examples: {correct}, total examples: {nparr.shape[0]}, accuracy = {correct/nparr.shape[0]}")

test = pd.read_csv("a1a_test.csv", header=None)
test_arr=test.to_numpy()

correct = 0
for j in range(test_arr.shape[0]):
  testVal = evaluate(t, test_arr, j)
  #print(f"evaluated: {testVal}")
  if testVal == True:
    correct += 1

print("Evaluate Testing Data")
print(f"total correct examples: {correct}, total examples: {test_arr.shape[0]}, accuracy = {correct/test_arr.shape[0]}")

Evaluate Training Data
total correct examples: 882, total examples: 1558, accuracy = 0.5661103979460848
Evaluate Testing Data
total correct examples: 16225, total examples: 30956, accuracy = 0.5241310246801912


#Limiting Depth, 5-fold cross-validation

In [251]:
#create data sets
fold1 = pd.read_csv("fold1.csv", header=None)
fold2 = pd.read_csv("fold2.csv", header=None)
fold3 = pd.read_csv("fold3.csv", header=None)
fold4 = pd.read_csv("fold4.csv", header=None)
fold5 = pd.read_csv("fold5.csv", header=None)

#1234 - 5
set1234 = pd.concat([fold1, fold2, fold3, fold4], ignore_index = True)
#1235 - 4
set1235 = pd.concat([fold1, fold2, fold3, fold5], ignore_index = True)
#1245 - 3
set1245 = pd.concat([fold1, fold2, fold5, fold4], ignore_index = True)
#1345 - 2
set1345 = pd.concat([fold1, fold5, fold3, fold4], ignore_index = True)
#2345 - 1
set2345 = pd.concat([fold5, fold2, fold3, fold4], ignore_index = True)

print(set1234)

      0    1    2    3    4    5    6    ...  117  118  119  120  121  122  123
0      -1    0    0    0    0    1    1  ...    0    0    0    0    0    0    0
1      -1    0    0    0    1    0    0  ...    0    0    0    0    0    0    0
2       1    0    0    0    0    1    0  ...    0    0    0    0    0    0    0
3      -1    0    0    0    1    0    1  ...    0    0    0    0    0    0    0
4      -1    1    0    0    0    0    1  ...    0    0    0    0    0    0    0
...   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...
1243   -1    0    0    1    0    0    1  ...    0    0    0    0    0    0    0
1244   -1    0    1    0    0    0    1  ...    0    0    0    0    0    0    0
1245   -1    0    1    0    0    0    1  ...    0    0    0    0    0    0    0
1246    1    0    0    0    0    1    0  ...    0    0    0    0    0    0    0
1247   -1    0    1    0    0    0    1  ...    0    0    0    0    0    0    0

[1248 rows x 124 columns]


In [ ]:
def limited_tree(df, tree=None, targetDepth, currentDepth):
  #unable to implement
  return